In [1]:
from dotenv import load_dotenv
from typing import Literal
from langchain_openai import ChatOpenAI
# from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import MessagesState, StateGraph,END
from langgraph.prebuilt import ToolNode
from composio_langgraph import Action, ComposioToolSet, App
import json

import os
load_dotenv(
)
openai_api_key=os.getenv('openai_api_key')
composio_api_key=os.getenv('composio_api_key')
google_api_key=os.getenv('google_api_key')


In [2]:
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

In [34]:

class Composio_agent:
    def __init__(self,tools:list):
       
       self.agent=self.setup_agent(tools)
       

    def setup_agent(self,tools:list):
        """
        This function is used to setup the agent
        Args:
            tools: list of tools from composio
        Returns:
            app: the agent
        """
        llm = ChatOpenAI(api_key=openai_api_key)
        tool_node = ToolNode(tools)

        model_with_tools = llm.bind_tools(tools)

        def call_model(state: MessagesState):
            """
            Process messages through the LLM and return the response
            """
            messages = state["messages"]
            response = model_with_tools.invoke(messages)
            return {"messages": [response]}

        workflow = StateGraph(MessagesState)
        workflow.add_node("agent", call_model)
        workflow.add_node("tools", tool_node)
        workflow.add_edge("__start__", "agent")
        workflow.add_edge("agent", "tools")
        workflow.add_edge("tools", END)
        app = workflow.compile()
        return app

    def chat(self,query:str):  

        """
        This tool is an agent used to interact with the agent it has a list of functionnalities that can be used to interact with the agent
        simply pass the query to the tool with the necessary information like ids if needed and it will return the response
        Args:
            query: str simply pass the query to the tool
        Returns:
            res: str
        """
        res=self.agent.invoke(
            {
                "messages": [
                (
                    "human",
                    query,
                )
            ]
        }
    )
        return res

    def display_graph(self):
        return display(
                        Image(
                                self.agent.get_graph().draw_mermaid_png(
                                    draw_method=MermaidDrawMethod.API,
                                )
                            )
                        )


In [10]:
import nest_asyncio
nest_asyncio.apply()
from pydantic_ai import Agent
from pydantic_ai.models.gemini import GeminiModel
from pydantic_ai.providers.google_gla import GoogleGLAProvider
from pydantic_ai.messages import ModelMessage
from dataclasses import dataclass

pydantic_llm=GeminiModel('gemini-2.0-flash', provider=GoogleGLAProvider(api_key=google_api_key))

In [32]:
toolset=ComposioToolSet(api_key=composio_api_key)
tools=toolset.get_tools(apps=[App.GMAIL])

c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\Cortana\.venv\Lib\site-packages\composio\client\collections.py:1183: UserWarning: Using all actions of an app is not recommended for production.Learn more: https://docs.composio.dev/patterns/tools/use-tools/use-specific-actions


  warnings.warn(


In [37]:
#get tool schemas
toolset.get_action_schemas(apps=[App.GMAIL])

[ActionModel(name='GMAIL_DELETE_DRAFT', description="Delete an email draft using gmail's api.", parameters=ActionParametersModel(properties={'draft_id': {'description': 'The ID of the draft to be deleted. Note: This is not the ID of the draft message that was created. Please provide a value of type string. This parameter is required.', 'title': 'Draft Id', 'type': 'string'}, 'user_id': {'default': 'me', 'description': "The user's email address or 'me' for the authenticated user. Please provide a value of type string.", 'title': 'User Id', 'type': 'string'}}, title='DeleteDraftRequest', type='object', required=['draft_id'], examples=None), response=ActionResponseModel(properties={'data': {'description': 'Data from the action execution. This parameter is required.', 'properties': {'success': {'description': 'Indicates if the draft was successfully deleted.', 'title': 'Success', 'type': 'boolean'}}, 'required': ['success'], 'title': 'Data', 'type': 'object'}, 'error': {'anyOf': [{'type': 

In [35]:
gmail_agent=Composio_agent(tools)

In [6]:
@dataclass
class Message_state:
    messages: list[ModelMessage]

In [12]:
def gmail_tool(query:str):
    """
    This tool is used to interact with the gmail agent
    Args:
        query: str
    Returns:
        res: str
    """
    return gmail_agent.chat(query)

In [13]:
agent=Agent(pydantic_llm, tools=[gmail_tool], instructions="you the gmail assistant, you are given a list of fuctionalities that the tool has access to to answer the user's query")

In [14]:
memory=Message_state(messages=[])


In [20]:
res=agent.run_sync(f'functionnalities:{},query: what is the first one', message_history=memory.messages)
memory.messages=res.all_messages()


In [26]:
gmail_agent.tools.get

StructuredTool(name='GMAIL_DELETE_DRAFT', description="Delete an email draft using gmail's api.", args_schema=<class 'composio.utils.shared.DeleteDraftRequest'>, handle_tool_error=True, handle_validation_error=True, func=<function ComposioToolSet._wrap_action.<locals>.function at 0x000001BA25A916C0>)

In [21]:
print(res.output)

The first tool is GMAIL\_DELETE\_DRAFT, which allows you to delete an email draft using Gmail's API.


In [19]:
memory.messages

[ModelRequest(parts=[UserPromptPart(content='functionnalities:[StructuredTool(name=\'GMAIL_DELETE_DRAFT\', description="Delete an email draft using gmail\'s api.", args_schema=<class \'composio.utils.shared.DeleteDraftRequest\'>, handle_tool_error=True, handle_validation_error=True, func=<function ComposioToolSet._wrap_action.<locals>.function at 0x000001BA25A916C0>), StructuredTool(name=\'GMAIL_DELETE_MESSAGE\', description="Delete an email message using gmail\'s api. note: this action requires the integration to have the `https://mail.google.com/` scope.", args_schema=<class \'composio.utils.shared.DeleteMessageRequest\'>, handle_tool_error=True, handle_validation_error=True, func=<function ComposioToolSet._wrap_action.<locals>.function at 0x000001BA25A91800>), StructuredTool(name=\'GMAIL_FETCH_EMAILS\', description=\'Action to fetch all emails from gmail.\', args_schema=<class \'composio.utils.shared.FetchEmailsRequest\'>, handle_tool_error=True, handle_validation_error=True, func=<

In [109]:
res =res['messages'][-1].model_dump_json()

In [110]:
res

'{"content":"{\\"data\\": {\\"calendars\\": [{\\"accessRole\\": \\"owner\\", \\"backgroundColor\\": \\"#9fe1e7\\", \\"colorId\\": \\"14\\", \\"conferenceProperties\\": {\\"allowedConferenceSolutionTypes\\": [\\"hangoutsMeet\\"]}, \\"defaultReminders\\": [{\\"method\\": \\"popup\\", \\"minutes\\": 30}], \\"etag\\": \\"\\\\\\"1689095734000000\\\\\\"\\", \\"foregroundColor\\": \\"#000000\\", \\"id\\": \\"padioutristan@gmail.com\\", \\"kind\\": \\"calendar#calendarListEntry\\", \\"notificationSettings\\": {\\"notifications\\": [{\\"method\\": \\"email\\", \\"type\\": \\"eventCreation\\"}, {\\"method\\": \\"email\\", \\"type\\": \\"eventChange\\"}, {\\"method\\": \\"email\\", \\"type\\": \\"eventCancellation\\"}, {\\"method\\": \\"email\\", \\"type\\": \\"eventResponse\\"}]}, \\"primary\\": true, \\"selected\\": true, \\"summary\\": \\"padioutristan@gmail.com\\", \\"timeZone\\": \\"America/New_York\\"}, {\\"accessRole\\": \\"reader\\", \\"backgroundColor\\": \\"#16a765\\", \\"colorId\\": \\

In [111]:
resto=json.loads(res)
res=json.loads(resto['content'])


In [112]:
res['data']['messages']

KeyError: 'messages'